## Homework 4
### BIOSTAT 257
### Joanna Boland
### May 29, 2020

### Question 1: Derivatives

### Question 2: Objective and Gradient Evaluator for a Single Datum


In [2]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathProgBase, MixedModels
using Random, RCall, Revise

┌ Info: Precompiling MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316]
└ @ Base loading.jl:1273
ERROR: LoadError: NLopt not installed properly; run Pkg.build("NLopt"), restart Julia, and try again.
Stacktrace:
 [1] error(::String) at .\error.jl:33
 [2] top-level scope at C:\Users\jbola\.julia\packages\NLopt\eqN9a\src\NLopt.jl:16
 [3] include at .\boot.jl:328 [inlined]
 [4] include_relative(::Module, ::String) at .\loading.jl:1105
 [5] include(::Module, ::String) at .\Base.jl:31
 [6] top-level scope at none:2
 [7] eval at .\boot.jl:330 [inlined]
 [8] eval(::Expr) at .\client.jl:425
 [9] top-level scope at .\none:3
in expression starting at C:\Users\jbola\.julia\packages\NLopt\eqN9a\src\NLopt.jl:15
ERROR: LoadError: Failed to precompile NLopt [76087f3c-5699-56af-9a33-bf431cd00edd] to C:\Users\jbola\.julia\compiled\v1.3\NLopt\faRdv_gzmkA.ji.
Stacktrace:
 [1] error(::String) at .\error.jl:33
 [2] compilecache(::Base.PkgId, ::String) at .\loading.jl:1283
 [3] _require(::Base.PkgId) at .\lo

ErrorException: Failed to precompile MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316] to C:\Users\jbola\.julia\compiled\v1.3\MixedModels\tBiYK_gzmkA.ji.

In [1]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q, 
        xtx, ztx, ztz, storage_qq)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        sleep(1e-3) # pretend this step takes 1ms
    end    
    ###################
    # Return
    ###################        
    return logl    
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [2]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

UndefVarError: UndefVarError: Random not defined